# Reverse Geocoding
Phase 2, 2018

The module censusgeocode was used to reverse geocode coordinates to census tracts and blocks. The dataframe is partitioned to allow for processing and saving in chunks.

Next steps:
- parallel processing to speed up geocoding

In [219]:
# Import modules
import os
import pandas as pd
import numpy as np
import censusgeocode as cg
from math import ceil
import time
import tqdm
import xml.etree.ElementTree as ET
import urllib.request
import xmltodict
from math import isnan

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Establish file paths
ROOT_dir = os.path.abspath('')
dataFolder = ROOT_dir + '/data'
outputFolder = ROOT_dir + '/phase2_output/geocodes'

In [ ]:
# Load CSV in to a dataframe
raw = pd.read_csv(dataFolder + '/2009-2014_RedCross_DisasterCases.csv',
                 encoding = "ISO-8859-1",
                 error_bad_lines = False)

In [196]:
# Keep single, multi fires and fires

# Build regex
def converter(list):
    output=""
    for letter in list:
        if letter == ' ':
            output += '[ ]'
        else:
            output += "[" + letter.upper() + letter.lower() + "]"
    return output

idx = np.where(raw['event_type_old_categories'].str.contains(converter('Fire : Single Family'), na = False)
              | raw['event_type_old_categories'].str.contains(converter('Fire : Multi-Family'), na = False)
              | raw['event_type_old_categories'].str.match(converter('Fire') + '$', na = False))

raw_fires = raw.loc[raw.index[idx[0]]].reset_index()

In [211]:
# Create dataframe for coordinate & census data
coords_df = raw_fires[['esri_longitude_x', 'esri_latitude_x']]
coords_df = coords_df.rename(columns = {'esri_longitude_x':'long', 'esri_latitude_x':'lat'})
#coords_df = coords_df.dropna(subset = ['long', 'lat'])

coords_df['tract'] = np.nan
coords_df['geoType'] = np.nan
coords_df['stateFips'] = np.nan
coords_df['name'] = np.nan

print('Original:',raw.shape)
print('Processed:',coords_df.shape)

coords_df.head()

Original: (566772, 41)
Processed: (464162, 6)


,long,lat,tract,geoType,stateFips,name
0,-86.765284,34.810583,NaN,NaN,NaN,NaN
1,-86.147937,31.764383,NaN,NaN,NaN,NaN
2,-85.622874,32.807411,NaN,NaN,NaN,NaN
3,-86.396463,34.617060,NaN,NaN,NaN,NaN
4,-86.777434,33.738204,NaN,NaN,NaN,NaN


In [230]:
# Partition data into chunks to allow for processing & saving in chunks

num_chunks = 100
#idx = np.where(coords_df['long'].notnull() & coords_df['lat'].notnull())[0]
chunk_size = ceil(coords_df.shape[0] / float(num_chunks))
chunk_idx = []
for i in range(num_chunks + 2):
    if i == 0:
        chunk_idx.append([0,chunk_size])
    else:
        temp = range(max(chunk_idx[-1]) + 1, max(chunk_idx[-1]) + chunk_size)
        if max(temp) > coords_df.shape[0]:
            chunk_idx.append([min(temp),coords_df.shape[0]])
        else:
            chunk_idx.append([min(temp),max(temp)])

In [ ]:
# Reverse Geocode using censusgeocode module
url_base = 'http://www.broadbandmap.gov/broadbandmap/census/tract?latitude='

start_time = time.time()
loopTime = []
failed = []
for chunk in tqdm.tqdm(range(len(chunk_idx[10:]))):
    idx_list = list(range(chunk_idx[chunk][0], chunk_idx[chunk][1] + 1))
    for idx in idx_list:
        start_time2 = time.time()
        lat = coords_df.loc[idx,'lat']
        long = coords_df.loc[idx,'long']
        if (isnan(lat) | isnan(long)):
            failed.append(idx)
        else:
            url = url_base + str(lat) + '&longitude=' + str(long) + '&format=xml'
            response = urllib.request.urlopen(url)
            xml = response.read()
            temp = xmltodict.parse(xml)

            if 'No Census Tract results found' not in list(temp['Response'].values()):
                temp = dict(temp['Response']['Results']['censusTract'])
                coords_df.loc[idx,'tract'] = temp['fips']
                coords_df.loc[idx,'geoType'] = temp['geographyType']
                coords_df.loc[idx,'stateFips'] = temp['stateFips']
                coords_df.loc[idx,'name'] = temp['name']
            else:
                failed.append(idx)
        loopTime.append(time.time() - start_time2)
    
    # Overwrite CSV
    coords_df.to_csv(outputFolder + '/geocodes_xml_method.csv')

print('Elapsed time:',time.time() - start_time)


  0%|          | 0/92 [00:00<?, ?it/s]
